# 3.3.3.2 Realistic simulation setup
Obtain the centroid for each chessboard 

In [40]:
vscode = 1


In [41]:
if(vscode == 0):
    # for vscode
    # for jupyter notebook
    from mpl_toolkits.mplot3d import axes3d
    import matplotlib.pyplot as plt

    %matplotlib notebook
elif(vscode == 1):
    %matplotlib qt
    

In [42]:
import matplotlib.pyplot as plt
import numpy as np
import cv2
 
from typing import Sequence
from calib_lib import *
from matplotlib.collections import PatchCollection      # pathcoletion for multiple patches
import matplotlib.cm as cm                              # for colormaps
from matplotlib.patches import Rectangle                # for rectangle representation
from matplotlib.patches import Ellipse                  # for ellipse representation   
from pixelization_lib import *   
#from ellipse import LsqEllipse                          # pip install lsq-ellipse


DECIMALS = 2            # how many decimal places to use in print


In [43]:
import logging 
import numpy as np
from skimage.measure import EllipseModel
from matplotlib.patches import Ellipse
import matplotlib.pyplot as plt

In [44]:
F = 16                                       # focal length( in mm )
gridSize = 1024
subpixel_grid = 128
image_size = np.array([1936,1216])               # sensor size(in pixels)
#image_size = np.array([11.345,7.126])
pixel_width = 5.86e-3                       # pixel size in mm
PX= image_size[0]/2.0                       # principal point_i x-coordinate
PY= image_size[1]/2.0                       # principal point_i y-coordinate
IMAGE_HEIGTH = image_size[1]
IMAGE_WIDTH = image_size[0]
THETA_X = 0                                 # roll angle
THETA_Y = 0                                 # pitch angle
THETA_Z = 0                                 # yaw angle

# camera Right
THETA_X_R = 0                                 # roll angle
THETA_Y_R = 0                                 # pitch angle
THETA_Z_R= 0                                 # yaw angle
# camera Left
THETA_X_L = 0                                 # roll angle
THETA_Y_L = 0                                 # pitch angle
THETA_Z_L= 0                                 # yaw angle

C_L = np.array([0,0,0])                     # camera centre
C_R = np.array([500,0,0])



In [45]:
# from 1 meter to 300 meters in steps of 10 meters
zdistance = np.arange(0,320000,1000)
zdistance[0] = 1000
xdistance = np.ones((zdistance.shape[0],))*250
ydistance = np.ones((zdistance.shape[0],))*10
X =np.array([xdistance,ydistance,zdistance]).T


x_zhang_L,x_zhang_R,X_arr_L,X_arr_R,x0_x,x0_y= create_1024_circle(X,PX,PY,thetax=THETA_X_L,thetay = THETA_Y_L,thetaz = THETA_Z_L,trans_x= -C_L[0],trans_y= -C_L[1],trans_z= -C_L[2],F = F,mx = (1/pixel_width),my =(1/pixel_width), npoints = 1024,radius = 10)

In [46]:
fig = plt.figure()
ax = fig.add_subplot(projection = '3d')
for i in range(10):
    ax.plot3D(X_arr_L[i,:,0],X_arr_L[i,:,1],X_arr_L[i,:,2])

In [47]:
fig = plt.figure()
ax = fig.add_subplot(121)
ax2 = fig.add_subplot(122)
for i in range(zdistance.shape[0]):
    ax.plot(x_zhang_L[i,:,0],x_zhang_L[i,:,1])
    ax2.plot(x_zhang_R[i,:,0],x_zhang_R[i,:,1])
ax.set_aspect(1)
ax2.set_aspect(1)

In [48]:
def spot_center_coord(cx0,cy0,pixelGridX,pixelGridY):
    cx0_max = np.where((cx0 < pixelGridX))
    cx0_min = np.where((cx0 > pixelGridX))
    #print(cx0_max[0][0])
    #print(cx0_min[0][-1])
    cy0_max = np.where((cy0 < pixelGridY))
    cy0_min = np.where((cy0 > pixelGridY))
    #print(cy0_max[0][0])
    #print(cy0_min[0][-1])

    return cx0_max[0][0],cx0_min[0][-1],cy0_max[0][0],cy0_min[0][-1]


def findcircle(x_arr):
    """
    Based on the image coordinates, and knowing it forms a square/rectangle, finds the centroid and radius

    Parameters
    ----------
    x_arr:  np.array
        image coordinates
    Returns
    -------
    cx0:    np.float
        centroid x
    cy0:    np.float
        centroid y    
    r0 : np.float
        circle radius
    """  
    
    Ax  = x_arr[0,0]
    Ay  = x_arr[0,1]
    Bx  = x_arr[int(x_arr.shape[0]/4),0]
    By  = x_arr[int(x_arr.shape[0]/4),1]
    Cx  = x_arr[int(x_arr.shape[0]/2),0]
    Cy  = x_arr[int(x_arr.shape[0]/2),1]
    Dx  = x_arr[int((x_arr.shape[0]/4*3)-1),0]
    Dy  = x_arr[int((x_arr.shape[0]/4*3)-1),1]


    cx0 = np.sum(x_arr[:,0])/np.sum(x_arr[:,0].shape[0])
    cy0 = np.sum(x_arr[:,1])/np.sum(x_arr[:,1].shape[0])

    r0 = np.abs((Dy-By)/2)
    
    return cx0,cy0,r0

def createcircle(x_arr,nrpoints = 100):
    """
    create the circle, using the function findcircle()

    Parameters
    ----------
    x_arr:  np.array
        image coordinates
    nrpoints: integer
        number of points that compose the circle
    Returns
    -------
    circle_x:    np.array
        points that define the circle in the cx0 coordinate
    circle_y:    np.array
        points that define the circle in the cy0 coordinate
    cx0:    np.float
        centroid x
    cy0:    np.float
        centroid y    
    r0 : np.float
        circle radius    
    """    

    cx0,cy0,r0 = findcircle(x_arr)

    theta = np.linspace(0, 2*np.pi, nrpoints)
    x = r0*np.cos(theta)
    y = r0*np.sin(theta)

    circle_x = x+cx0
    circle_y = y+cy0


    return circle_x, circle_y,cx0,cy0,r0

In [49]:
"""
NMI : 2021-11-11 
Minimum Volume Enclosing Ellipsoids, see e.g.
NIMA MOSHTAGH : MINIMUM VOLUME ENCLOSING ELLIPSOIDS
or 
Linus Källberg : Minimum_Enclosing_Balls_and_Ellipsoids (Thesis)
"""
from warnings import warn

from numpy import pi
from numpy import sqrt
from numpy import arccos
from numpy import dot, outer
from numpy import diag, transpose
from numpy import append
from numpy import asarray
from numpy import ones
from numpy import argmax

from numpy.linalg import inv
from numpy.linalg import norm
from numpy.linalg import eig


def mvee( data, tolerance=1e-4, maxcnt=10000 ):
    """
    param data: list of xy data points
    param tolerance: termination condition for iterative approximation
    param maxcnt: maximum number of iterations
    type data: iterable of float
    type tolerance: float
    return: (offset, semiaxis, angle)
    return type: ( (float, float), (float, float), float )
    """
    locdata = asarray( data )
    N = len( locdata )
    if not locdata.shape == ( N, 2):
        raise ValueError ( " data must be of shape( n, 2 )" )
    if tolerance >= 1 or tolerance <= 0:
        raise ValueError (" 0 < tolerance < 1 required")
    if not isinstance( maxcnt, int ):
        raise TypeError
    if not maxcnt > 0:
        raise ValueError
    count = 1
    err = 1
    d = 2
    d1 = d + 1
    u = ones( N ) / N
    P = transpose( locdata )
    Q = append( P, ones( N ) ).reshape( 3, -1 )
    while ( err > tolerance):
        X = dot( Q, dot( diag( u ), transpose( Q ) ) )
        M = diag( 
            dot( 
                transpose( Q ),
                dot(
                    inv( X ),
                    Q
                )
            )
        )
        maximum = max( M )
        j = argmax( M )
        step_size = ( maximum - d1 ) / ( d1 * ( maximum - 1 ) )
        new_u = ( 1 - step_size ) * u
        new_u[ j ] += step_size
        err = norm( new_u - u )
        count = count + 1
        u = new_u
        if count > maxcnt:
            warn(
                "Process did not converge in {} steps".format(
                    count - 1
                ),
                UserWarning
            )
            break
    U = diag( u )
    c = dot( P,  u )
    A = inv(
        dot(
            P,
            dot( U, transpose( P ) )
        ) - outer( c, c )
    ) / d
    E, V = eig( A )
    phiopt = arccos( V[ 0, 0 ] )
    if V[ 0, 1 ] < 0: 
        phiopt = 2 * pi - phiopt
    ### cw vs ccw and periodicity of pi
    phiopt = -phiopt % pi
    sol =  (  c, sqrt( 1.0 / E ), phiopt)
    return sol

In [50]:
x0_L = x_zhang_L[:,:,0]
y0_L = x_zhang_L[:,:,1]


x0_R = x_zhang_R[:,:,0]
y0_R = x_zhang_R[:,:,1]

### Find the center of the circle(point)

In [51]:
# Generate pixel grid
pixelGridX  = (np.arange(0,image_size[0])) 
pixelGridY  = (np.arange(0,image_size[1]))
pixelGridXX, pixelGridYY = np.meshgrid(pixelGridX,pixelGridY)

cx0_L = np.zeros((x_zhang_L.shape[0],1))
cy0_L = np.zeros((x_zhang_L.shape[0],1))
r0_L = np.zeros((x_zhang_L.shape[0],1))
cx0_R = np.zeros((x_zhang_R.shape[0],1))
cy0_R = np.zeros((x_zhang_R.shape[0],1))
r0_R = np.zeros((x_zhang_R.shape[0],1))

#
x0_cL = np.zeros((x_zhang_L.shape[0],gridSize))
y0_cL = np.zeros((x_zhang_L.shape[0],gridSize))
x0_cR = np.zeros((x_zhang_R.shape[0],gridSize))
y0_cR = np.zeros((x_zhang_R.shape[0],gridSize))


for point_i in range(x0_L.shape[0]):

    #circle definition 
    # multiply by pixel_width to get the results in mm, not in pixels
    x0_cL[point_i,:],y0_cL[point_i,:],cx0_L[point_i,:],cy0_L[point_i,:],r0_L[point_i,:] = createcircle(x_zhang_L[point_i,:],gridSize)
    x0_cR[point_i,:],y0_cR[point_i,:],cx0_R[point_i,:],cy0_R[point_i,:],r0_R[point_i,:] = createcircle(x_zhang_R[point_i,:],gridSize)

## Find the centroids of every ellipse and circle using the moment method

In [52]:
# get for 512 per spot
gridSize = 1024


# create the arrays to store the centroids
momentL = np.zeros((x0_L.shape[0],2))
momentR = np.zeros((x0_R.shape[0],2))

#for each chessboard
for point_i in range(x0_L.shape[0]):
    momentL[point_i,0] = np.sum(x0_L[point_i,:])/np.sum(x0_L.shape[1])
    momentL[point_i,1] = np.sum(y0_L[point_i,:])/np.sum(y0_L.shape[1])

    momentR[point_i,0] = np.sum(x0_R[point_i,:])/np.sum(x0_R.shape[1])
    momentR[point_i,1] = np.sum(y0_R[point_i,:])/np.sum(y0_R.shape[1])

In [53]:
np.sum(x0_L[0,:])/np.sum(x0_L.shape[1])

1650.62060546875

### Difference between original point without spot and centroiding the spot

In [54]:
coordinates_orig_L = (np.array([cx0_L,cy0_L]).T).reshape((zdistance.shape[0],2))
coordinates_orig_R = (np.array([cx0_R,cy0_R]).T).reshape((zdistance.shape[0],2))

In [55]:
np.sum(momentL[:,:]-(coordinates_orig_L[:,:]))

0.0

In [92]:
np.sum(momentR[:,:]-(coordinates_orig_R[:,:]))

0.0

### Find max and mim values that create boundaries in the ellipse and plot the center of the ellipse 
Also, meshgrid is added to the ellipse, representing the pixels

In [57]:
x_L_lst = []
y_L_lst = []

#fig = plt.figure()
#ax = fig.add_subplot()
for point in range(x0_L.shape[0]):
    ax.plot(x0_L[point,:],y0_L[point,:])
    max_pt_x_L = np.max(x_zhang_L[point,:,0])
    max_pt_y_L = np.max(x_zhang_L[point,:,1])
    min_pt_x_L = np.min(x_zhang_L[point,:,0])
    min_pt_y_L = np.min(x_zhang_L[point,:,1])

    pts = np.array([[max_pt_x_L,max_pt_y_L],[max_pt_x_L,min_pt_y_L],[min_pt_x_L,min_pt_y_L],[min_pt_x_L,max_pt_y_L]])
    cmin_pxl_L = np.zeros((pts.shape[0],pts.shape[1]))
    for pts_i,pts_d in enumerate(pts):
        cx0max_L,cx0min_L,cy0max_L,cy0min_L  = spot_center_coord(pts_d[0],pts_d[1],pixelGridX,pixelGridY)

        # max and min points in (x,y) coordinates
        cmin_pxl_L[pts_i,0] = (cx0min_L)
        cmin_pxl_L[pts_i,1] = (cy0min_L)

    x_L_lst.append(np.arange(cmin_pxl_L[3,0],cmin_pxl_L[0,0]+1,1))
    y_L_lst.append(np.arange(cmin_pxl_L[1,1],cmin_pxl_L[0,1]+1,1))
    xx,yy = np.meshgrid(x_L_lst[point],y_L_lst[point])

#ax.plot(xx,yy, color = 'g',ls = "None", marker = "*")

#ax.set_aspect(1)    
#plt.show()

In [58]:
x_R_lst = []
y_R_lst = []

for point in range(x0_R.shape[0]):
    #ax.plot(x0_L[chess_b_i,point,:],y0_L[chess_b_i,point,:])
    max_pt_x_R = np.max(x_zhang_R[point,:,0])
    max_pt_y_R = np.max(x_zhang_R[point,:,1])
    min_pt_x_R = np.min(x_zhang_R[point,:,0])
    min_pt_y_R = np.min(x_zhang_R[point,:,1])

    pts = np.array([[max_pt_x_R,max_pt_y_R],[max_pt_x_R,min_pt_y_R],[min_pt_x_R,min_pt_y_R],[min_pt_x_R,max_pt_y_R]])
    cmin_pxl_R = np.zeros((pts.shape[0],pts.shape[1]))
    for pts_i,pts_d in enumerate(pts):
        cx0max_R,cx0min_R,cy0max_R,cy0min_R  = spot_center_coord(pts_d[0],pts_d[1],pixelGridX,pixelGridY)

        # max and min points in (x,y) coordinates
        cmin_pxl_R[pts_i,0] = (cx0min_R)
        cmin_pxl_R[pts_i,1] = (cy0min_R)

    x_R_lst.append(np.arange(cmin_pxl_R[3,0],cmin_pxl_R[0,0]+1,1))
    y_R_lst.append(np.arange(cmin_pxl_R[1,1],cmin_pxl_R[0,1]+1,1))
    xx,yy = np.meshgrid(x_R_lst[point],y_R_lst[point])

    #ax.plot(xx,yy, color = 'g',ls = "None", marker = "*")
#ax.set_aspect(1)    
#plt.show()

### Find the ellipses center, semi-major and semi-minor

In [59]:
X_R = np.array(x_zhang_R[:,:,:])
X_L = np.array(x_zhang_L[:,:,:])

xy_R_arr = np.zeros((x_zhang_R.shape[0],2))
ab_R_arr = np.zeros((x_zhang_R.shape[0],2))
theta_arr_R = np.zeros((x_zhang_R.shape[0],1))

xy_L_arr = np.zeros((x_zhang_L.shape[0],2))
ab_L_arr = np.zeros((x_zhang_L.shape[0],2))
theta_arr_L = np.zeros((x_zhang_L.shape[0],1))

# centroid array
for point_i in range(x_zhang_L.shape[0]):
    xy_L_arr[point_i,:], ab_L_arr[point_i,:],theta_arr_R[point_i,:] = mvee(X_L[point_i,:,:])
    xy_R_arr[point_i,:], ab_R_arr[point_i,:],theta_arr_L[point_i,:] = mvee(X_R[point_i,:,:])


KeyboardInterrupt: 

In [71]:
"""np.save("xy_centroid_estimation_L.npy",xy_L_arr)
np.save("xy_centroid_estimation_R.npy",xy_R_arr)
np.save("ab_L_arr.npy",ab_L_arr)
np.save("ab_R_arr.npy",ab_R_arr)
np.save("theta_arr_L.npy",theta_arr_L)
np.save("theta_arr_R.npy",theta_arr_R)"""


'np.save("xy_centroid_estimation_L.npy",xy_L_arr)\nnp.save("xy_centroid_estimation_R.npy",xy_R_arr)\nnp.save("ab_L_arr.npy",ab_L_arr)\nnp.save("ab_R_arr.npy",ab_R_arr)\nnp.save("theta_arr_L.npy",theta_arr_L)\nnp.save("theta_arr_R.npy",theta_arr_R)'

In [60]:
xy_L_arr = np.load("xy_centroid_estimation_L.npy")
xy_R_arr = np.load("xy_centroid_estimation_R.npy")
ab_L_arr = np.load("ab_L_arr.npy")
ab_R_arr = np.load("ab_R_arr.npy") 
theta_arr_L = np.load("theta_arr_L.npy")
theta_arr_R = np.load("theta_arr_R.npy")


### Do analysis pixel by pixel and check if correlates withthe original ellipse

In [61]:
def circlediscretization(X_crns_L,Y_crns_L,xgrid_L,ygrid_L,cx0_L,cy0_L,r0_L,point,subpixel_grid,pixel_width):
       
    lstL = []
    ptchsL = []

    for i in range(X_crns_L.flatten().shape[0]):
        cx0max_L = X_crns_L.flatten()[i]
        cy0max_L = Y_crns_L.flatten()[i]

        # translate the left bottom corner
        x_pixel_L = cx0max_L+xgrid_L
        y_pixel_L = cy0max_L+ygrid_L
        # create sub-pixel grid
        xx_pixel_L, yy_pixel_L = np.meshgrid(x_pixel_L,y_pixel_L)
        
        circ_L = np.sqrt((xx_pixel_L-cx0_L[point,:])**2 + (yy_pixel_L-cy0_L[point,:])**2) <= r0_L[point,:]
        circ_corners_L = np.where(circ_L == True)
        xxgrid_L = np.array(circ_corners_L).T


        lstL.append((xxgrid_L.shape[0]/(subpixel_grid**2))*100)

        ptchsL.append(Rectangle((cx0max_L,cy0max_L), pixel_width,pixel_width))

    return xxgrid_L,lstL,ptchsL

In [62]:
def ellipsediscretization(X_crns_L,Y_crns_L,xgrid_L,ygrid_L,xc_arr_L,yc_arr_L,a_arr_L,b_arr_L,theta_arr,point,subpixel_grid,pixel_width):
    """
    Based on the parameters fed to the function, it discretizes the ellipse returning the occupied percentage of each pixel by the ellipse
    ----------
    X_crns: np.array
        Coordinate mesh grid on the x axis
    Y_crns:    np.array   
        Coordinate mesh grid on the y axis 
    x_subgrid:    np.array
        x-axis grid for the subgrid portion
    y_subgrid:    np.array
        y-axis grid for the subgrid portion
    xc_arr
    yc_arr
    a_arr
    b_arr
    theta_arr
    chess
    point
    subpixel_grid
    pixel_width
    
    Returns    
    -------    
    world_pts_arr : np.array
        world coordinates
    x_zhang_R:    np.array
        right image coordinates
    x_zhang_L:    np.array
        Left image coordinates
    P_L : np.array
        Left camera Projection matrix
    P_R : np.array
        Right camera Projection matrix
                
    """
    
    lstL = []
    ptchsL = []

    for i in range(X_crns_L.flatten().shape[0]):
        cx0max_L = X_crns_L.flatten()[i]
        cy0max_L = Y_crns_L.flatten()[i]

        # translate the left bottom corner
        x_pixel_L = cx0max_L+xgrid_L
        y_pixel_L = cy0max_L+ygrid_L
        # create sub-pixel grid
        xx_pixel_L, yy_pixel_L = np.meshgrid(x_pixel_L,y_pixel_L)

        x_L, y_L = xx_pixel_L.flatten(), yy_pixel_L.flatten()
                
        # calculate whether a point is inside the ellipse or not
        cos_angle = np.cos(np.radians(180.-theta_arr*180/np.pi))
        sin_angle = np.sin(np.radians(180.-theta_arr*180/np.pi))

        x_c_L = x_L - xc_arr_L
        y_c_L = y_L - yc_arr_L

        xct_L = x_c_L * cos_angle - y_c_L * sin_angle
        yct_L = x_c_L * sin_angle + y_c_L * cos_angle 

        rad_cc = (xct_L**2/(a_arr_L)**2) + (yct_L**2/(b_arr_L)**2)

        # set the 
        xxgrid_L = np.where(rad_cc <= 1.)[0]


        lstL.append((xxgrid_L.shape[0]/(subpixel_grid**2))*100)

        ptchsL.append(Rectangle((cx0max_L,cy0max_L), 1,1))
    
    return xxgrid_L,lstL,ptchsL 

In [72]:

from matplotlib.path import Path


import warnings
warnings.filterwarnings("error")

centroid_L_arr = np.zeros((x0_L.shape[0],x0_L.shape[1],2))
xgrid_L = np.linspace(0,1,subpixel_grid)
ygrid_L = np.linspace(0,1,subpixel_grid)

chess_pts_L = []
chess_ptchs_L = []

centroid_L_arr = np.zeros((x0_L.shape[0],2))
# iterate the chessboards


for point_i in range(x0_L.shape[0]):
    
    lstL = []
    ptchsL = []
    
    # meshgrid for the pixel
    [X_crns_L,Y_crns_L] = np.meshgrid(x_L_lst[point_i],y_L_lst[point_i])

    xxgrid_L,lstL,ptchsL = circlediscretization(X_crns_L,Y_crns_L,xgrid_L,ygrid_L,cx0_L,cy0_L,r0_L,point_i,subpixel_grid,1)
    # if the max and min values as 0 it means the point was not correctly identified
    if(np.max(lstL) == 0 and np.min(lstL) == 0):
        xxgrid_L,lstL,ptchsL = circlediscretization(X_crns_L,Y_crns_L,xgrid_L,ygrid_L,cx0_L,cy0_L,r0_L,point_i,subpixel_grid,1)
                                                    

    centroid_L_x = np.round(np.sum(np.multiply(X_crns_L.flatten(),lstL))/(np.sum(lstL)) + 0.5,decimals = 0)
    centroid_L_y = np.round(np.sum(np.multiply(Y_crns_L.flatten(),lstL))/(np.sum(lstL)) + 0.5,decimals = 0)
    centroid_L_arr[point_i,:]  = (centroid_L_x,centroid_L_y)

    chess_pts_L.append(lstL)
    chess_ptchs_L.append(ptchsL)            


In [93]:
centroid_L_arr

array([[1651.,  635.],
       [1651.,  635.],
       [1309.,  622.],
       [1196.,  617.],
       [1139.,  615.],
       [1105.,  613.],
       [1082.,  613.],
       [1066.,  612.],
       [1053.,  611.],
       [1044.,  611.],
       [1036.,  611.],
       [1030.,  610.],
       [1025.,  610.],
       [1021.,  610.],
       [1017.,  610.],
       [1014.,  610.],
       [1011.,  610.],
       [1008.,  610.],
       [1006.,  610.],
       [1004.,  609.],
       [1002.,  609.],
       [1001.,  609.],
       [ 999.,  609.],
       [ 998.,  609.],
       [ 996.,  609.],
       [ 995.,  609.],
       [ 994.,  609.],
       [ 993.,  609.],
       [ 992.,  609.],
       [ 992.,  609.],
       [ 991.,  609.],
       [ 990.,  609.],
       [ 989.,  609.],
       [ 989.,  609.],
       [ 988.,  609.],
       [ 988.,  609.],
       [ 987.,  609.],
       [ 986.,  609.],
       [ 986.,  609.],
       [ 986.,  609.],
       [ 985.,  609.],
       [ 985.,  609.],
       [ 984.,  609.],
       [ 98

In [73]:
np.sum(centroid_L_arr[:,:] - (coordinates_orig_L[:320,:]))

-60.664306640625

In [75]:

warnings.filterwarnings("error")

centroid_R_arr = np.zeros((x0_R.shape[0],x0_R.shape[1],2))
xgrid_R = np.linspace(0,1,subpixel_grid)
ygrid_R = np.linspace(0,1,subpixel_grid)

chess_pts_R = []
chess_ptchs_R = []

centroid_R_arr = np.zeros((x0_R.shape[0],2))
# iterate the chessboards


for point_i in range(x0_R.shape[0]):
    
    lstR = []
    ptchsR = []
    
    # meshgrid for the pixel
    [X_crns_R,Y_crns_R] = np.meshgrid(x_R_lst[point_i],y_R_lst[point_i])


    xxgrid_R,lstR,ptchsR = circlediscretization(X_crns_R,Y_crns_R,xgrid_R,ygrid_R,cx0_R,cy0_R,r0_R,point_i,subpixel_grid,1)

    # if the max and min values as 0 it means the point was not correctly identified
    if(np.max(lstR) == 0 and np.min(lstR) == 0):
        xxgrid_R,lstR,ptchsR = circlediscretization(X_crns_R,Y_crns_R,xgrid_R,ygrid_R,cx0_R,cy0_R,r0_R,point_i,subpixel_grid,1)
                                                    

    centroid_R_x = np.round(np.sum(np.multiply(X_crns_R.flatten(),lstR))/(np.sum(lstR)) + 0.5,decimals=0)
    centroid_R_y = np.round(np.sum(np.multiply(Y_crns_R.flatten(),lstR))/(np.sum(lstR)) + 0.5,decimals = 0)
    centroid_R_arr[point_i,:]  = (centroid_R_x,centroid_R_y)

    chess_pts_R.append(lstR)
    chess_ptchs_R.append(ptchsR)            


In [94]:
centroid_R_arr

array([[285., 635.],
       [285., 635.],
       [627., 622.],
       [740., 617.],
       [797., 615.],
       [831., 613.],
       [854., 613.],
       [870., 612.],
       [883., 611.],
       [892., 611.],
       [900., 611.],
       [906., 610.],
       [911., 610.],
       [915., 610.],
       [919., 610.],
       [922., 610.],
       [925., 610.],
       [928., 610.],
       [930., 610.],
       [932., 609.],
       [934., 609.],
       [935., 609.],
       [937., 609.],
       [938., 609.],
       [940., 609.],
       [941., 609.],
       [942., 609.],
       [943., 609.],
       [944., 609.],
       [944., 609.],
       [945., 609.],
       [946., 609.],
       [947., 609.],
       [947., 609.],
       [948., 609.],
       [948., 609.],
       [949., 609.],
       [950., 609.],
       [950., 609.],
       [950., 609.],
       [951., 609.],
       [951., 609.],
       [952., 609.],
       [952., 609.],
       [952., 609.],
       [953., 609.],
       [953., 609.],
       [953.,

In [76]:
np.sum(centroid_R_arr[:,:] - (coordinates_orig_R[:,:]))

-22.76202392578125

# Matrixes

In [77]:
P_L_original_p = [np.array([[2.73037543e+03, 0.00000000e+00, 9.68000000e+02, 0.00000000e+00],
                        [0.00000000e+00, 2.73037543e+03, 6.08000000e+02, 0.00000000e+00],
                        [0.00000000e+00, 0.00000000e+00, 1.00000000e+00, 0.00000000e+00]]),
                np.array([[2.73037543e+03, 0.00000000e+00, 9.68000000e+02, 0.00000000e+00],
                        [0.00000000e+00, 2.73037543e+03, 6.08000000e+02, 0.00000000e+00],
                        [0.00000000e+00, 0.00000000e+00, 1.00000000e+00, 0.00000000e+00]]),
                np.array([[2.73037543e+03, 0.00000000e+00, 9.68000000e+02, 0.00000000e+00],
                        [0.00000000e+00, 2.73037543e+03, 6.08000000e+02, 0.00000000e+00],
                        [0.00000000e+00, 0.00000000e+00, 1.00000000e+00, 0.00000000e+00]]),
                np.array([[2.73037543e+03, 0.00000000e+00, 9.68000000e+02, 0.00000000e+00],
                        [0.00000000e+00, 2.73037543e+03, 6.08000000e+02, 0.00000000e+00],
                        [0.00000000e+00, 0.00000000e+00, 1.00000000e+00, 0.00000000e+00]]),
                np.array([[2.73037543e+03, 0.00000000e+00, 9.68000000e+02, 0.00000000e+00],
                        [0.00000000e+00, 2.73037543e+03, 6.08000000e+02, 0.00000000e+00],
                        [0.00000000e+00, 0.00000000e+00, 1.00000000e+00, 0.00000000e+00]]),
                np.array([[2.73037543e+03, 0.00000000e+00, 9.68000000e+02, 0.00000000e+00],
                        [0.00000000e+00, 2.73037543e+03, 6.08000000e+02, 0.00000000e+00],
                        [0.00000000e+00, 0.00000000e+00, 1.00000000e+00, 0.00000000e+00]])]                        

In [78]:
P_R_original_p =  [np.array([[ 2.73037543e+03,  0.00000000e+00,9.68000000e+02,-1.36518771e+06],
                        [ 0.00000000e+00,  2.73037543e+03,  6.08000000e+02,0.00000000e+00],
                        [ 0.00000000e+00,  0.00000000e+00,  1.00000000e+00,0.00000000e+00]]),
                np.array([[ 2.73037543e+03,  0.00000000e+00,  9.68000000e+02,-1.36518771e+06],
                        [ 0.00000000e+00,  2.73037543e+03,  6.08000000e+02,0.00000000e+00],
                        [ 0.00000000e+00,  0.00000000e+00,  1.00000000e+00,0.00000000e+00]]),
                np.array([[ 2.73037543e+03,  0.00000000e+00,  9.68000000e+02,-1.36518771e+06],
                        [ 0.00000000e+00,  2.73037543e+03,  6.08000000e+02,0.00000000e+00],
                        [ 0.00000000e+00,  0.00000000e+00,  1.00000000e+00,0.00000000e+00]]),
                np.array([[ 2.73037543e+03,  0.00000000e+00,  9.68000000e+02,-1.36518771e+06],
                        [ 0.00000000e+00,  2.73037543e+03,  6.08000000e+02,0.00000000e+00],
                        [ 0.00000000e+00,  0.00000000e+00,  1.00000000e+00,0.00000000e+00]]),
                np.array([[ 2.73037543e+03,  0.00000000e+00,  9.68000000e+02,-1.36518771e+06],
                        [ 0.00000000e+00,  2.73037543e+03,  6.08000000e+02,0.00000000e+00],
                        [ 0.00000000e+00,  0.00000000e+00,  1.00000000e+00,0.00000000e+00]]),
                np.array([[ 2.73037543e+03,  0.00000000e+00,  9.68000000e+02,-1.36518771e+06],
                        [ 0.00000000e+00,  2.73037543e+03,  6.08000000e+02,0.00000000e+00],
                        [ 0.00000000e+00,  0.00000000e+00,  1.00000000e+00,0.00000000e+00]])]


In [79]:
ret_S_2_p = [[0.48382956215306],
             [0.2480641466427782],
             [0.5044964792035515],
             [0.6493056088977223],
             [0.5858582529660523],
             [0.7349147342977488]]

ret_S_3_p = [[0.6444367035996724],
             [0.6974252709552935],
             [0.573508528650511],
             [0.665329407548191],
             [0.5727506628172644],
             [0.5066560423117088]]

ret_S_4_p = [[0.11495297071858447],
             [0.4353682960725999],
             [0.33533863107822903],
             [0.2932726745717978],
             [0.3074331716408665],
             [0.4381999154429301]]            

In [80]:
new_mtxL_list_2_p =     [[np.array([[3415.476927, 0.000000, 866.529480],
                                [0.000000, 3427.475550, 591.419726],
                                [0.000000, 0.000000, 1.000000]])],
                        [np.array([[2025.946993, 0.000000, 940.742322],
                                [0.000000, 2028.540029, 612.812453],
                                [0.000000, 0.000000, 1.000000]])],
                        [np.array([[2634.161432, 0.000000, 917.074509],
                                [0.000000, 2639.351298, 660.793013],
                                [0.000000, 0.000000, 1.000000]])],
                        [np.array([[2593.455177, 0.000000, 937.271354],
                                [0.000000, 2596.819522, 595.536887],
                                [0.000000, 0.000000, 1.000000]])],
                        [np.array([[2609.279406, 0.000000, 919.938333],
                                [0.000000, 2613.443863, 624.119490],
                                [0.000000, 0.000000, 1.000000]])],
                        [np.array([[2582.091025, 0.000000, 936.730455],
                                [0.000000, 2584.720309, 599.633110],
                                [0.000000, 0.000000, 1.000000]])]]  

new_mtxL_list_3_p =     [[np.array([[2.56358996e+03, 0.00000000e+00, 9.00413244e+02],
                                [0.00000000e+00, 2.55836987e+03, 5.95131659e+02],
                                [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]])],
                        [np.array([[2.57159779e+03, 0.00000000e+00, 8.79677473e+02],
                                [0.00000000e+00, 2.56448675e+03, 6.20432029e+02],
                                [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]])],
                        [np.array([[2.56141720e+03, 0.00000000e+00, 9.03328715e+02],
                                [0.00000000e+00, 2.55833609e+03, 6.11736763e+02],
                                [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]])],
                        [np.array([[2.56739511e+03, 0.00000000e+00, 9.07042600e+02],
                                [0.00000000e+00, 2.56017541e+03, 6.03841330e+02],
                                [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]])],
                        [np.array([[2546.944320, 0.000000, 925.728803],
                                [0.000000, 2545.877929, 594.959136],
                                [0.000000, 0.000000, 1.000000]])],
                        [np.array([[2543.759607, 0.000000, 920.626093],
                                [0.000000, 2543.153247, 593.669771],
                                [0.000000, 0.000000, 1.000000]])]]                                  

new_mtxL_list_4_p =     [[np.array([[2471.269294, 0.000000, 909.453470],
                                [0.000000, 2458.434752, 628.586125],
                                [0.000000, 0.000000, 1.000000]])],
                        [np.array([[2492.730191, 0.000000, 937.868012],
                                [0.000000, 2453.419376, 607.329803],
                                [0.000000, 0.000000, 1.000000]])],
                        [np.array([[2377.283484, 0.000000, 899.953008],
                                [0.000000, 2367.931216, 640.622319],
                                [0.000000, 0.000000, 1.000000]])],
                        [np.array([[2296.716257, 0.000000, 915.489948],
                                [0.000000, 2296.994123, 619.468668],
                                [0.000000, 0.000000, 1.000000]])],
                        [np.array([[2313.198140, 0.000000, 936.749683],
                                [0.000000, 2312.726330, 634.529867],
                                [0.000000, 0.000000, 1.000000]])],
                        [np.array([[2660.339510, 0.000000, 917.221163],
                                [0.000000, 2662.833935, 619.250302],
                                [0.000000, 0.000000, 1.000000]])]]  

In [81]:
ret_L_list_2_p =    [[0.19912488804572606],
                    [0.18191521222856677],
                    [0.4471677741963123],
                    [0.5865182028596742],
                    [0.529290181134825],
                    [0.669153090609754]]

ret_L_list_3_p =    [[0.6067080727105552],
                    [0.6574520960736969],
                    [0.5338728359408561],
                    [0.5742361284592391],
                    [0.48220078216376067],
                    [0.42299196053794874]]

ret_L_list_4_p =    [[0.08575734649873147],
                    [0.3167756093700254],
                    [0.26277033422482515],
                    [0.22998670447853434],
                    [0.21317684936216882],
                    [0.20820309169589335]]

In [82]:
new_mtxR_list_2_p =     [[np.array([[2387.502824, 0.000000, 962.211229],
                                [0.000000, 2394.962325, 712.916876],
                                [0.000000, 0.000000, 1.000000]])],
                        [np.array([[2322.950349, 0.000000, 989.618250],
                                [0.000000, 2326.393900, 642.560343],
                                [0.000000, 0.000000, 1.000000]])],
                        [np.array([[2600.070750, 0.000000, 1012.469898],
                                [0.000000, 2603.358796, 704.589934],
                                [0.000000, 0.000000, 1.000000]])],
                        [np.array([[2579.859915, 0.000000, 1022.673288],
                                [0.000000, 2579.272185, 626.014042],
                                [0.000000, 0.000000, 1.000000]])],
                        [np.array([[2594.716841, 0.000000, 1016.073982],
                                [0.000000, 2595.638264, 654.018781],
                                [0.000000, 0.000000, 1.000000]])],
                        [np.array([[2567.043553, 0.000000, 1023.351468],
                                [0.000000, 2565.861942, 629.353803],
                                [0.000000, 0.000000, 1.000000]])]]
                        
new_mtxR_list_3_p =     [[np.array([[ 9.97793158e-01,  6.38714110e-03, -6.60909825e-02],
                                [-6.44880157e-03,  9.99978947e-01, -7.19666375e-04],
                                [ 6.60849945e-02,  1.14428582e-03,  9.97813341e-01]])],
                        [np.array([[2.56835381e+03, 0.00000000e+00, 9.73932138e+02],
                                [0.00000000e+00, 2.56726682e+03, 6.45708931e+02],
                                [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]])],
                        [np.array([[2.55806726e+03, 0.00000000e+00, 9.81954052e+02],
                                [0.00000000e+00, 2.56132169e+03, 6.37192983e+02],
                                [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]])],
                        [np.array([[2.56343044e+03, 0.00000000e+00, 9.49654543e+02],
                                [0.00000000e+00, 2.56527639e+03, 6.29615996e+02],
                                [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]])],
                        [np.array([[2553.923857, 0.000000, 969.112990],
                                [0.000000, 2556.833857, 623.886753],
                                [0.000000, 0.000000, 1.000000]])],
                        [np.array([[2546.185286, 0.000000, 969.790541],
                                [0.000000, 2549.933375, 621.862121],
                                [0.000000, 0.000000, 1.000000]])]]

new_mtxR_list_4_p =     [[np.array([[2781.431909, 0.000000, 872.800124],
                                [0.000000, 2777.377444, 666.324585],
                                [0.000000, 0.000000, 1.000000]])],
                        [np.array([[2432.871800, 0.000000, 959.041214],
                                [0.000000, 2434.751553, 751.074855],
                                [0.000000, 0.000000, 1.000000]])],
                        [np.array([[2435.038898, 0.000000, 942.247178],
                                [0.000000, 2435.674880, 714.132299],
                                [0.000000, 0.000000, 1.000000]])],
                        [np.array([[2394.869886, 0.000000, 916.504805],
                                [0.000000, 2393.080034, 654.922188],
                                [0.000000, 0.000000, 1.000000]])],
                        [np.array([[2408.760172, 0.000000, 930.011570],
                                [0.000000, 2407.142251, 668.685677],
                                [0.000000, 0.000000, 1.000000]])],
                        [np.array([[2414.717710, 0.000000, 940.417138],
                                [0.000000, 2414.074344, 664.689731],
                                [0.000000, 0.000000, 1.000000]])]]                                

In [83]:
ret_R_list_2_p =    [[0.16997805220999498],      
                    [0.19263678452692068],
                    [0.4800029832407351],
                    [0.6257174143470249],
                    [0.5612022425113058],
                    [0.708122793907135]]

ret_R_list_3_p =    [[0.5919088397195084],      
                    [0.653565407611631],
                    [0.5401694328472426],
                    [0.5617705132959867],
                    [0.4744411668357235],
                    [0.4168205806424714]]

ret_R_list_4_p =    [[0.10576985026351769],      
                    [0.3000277156097531],
                    [0.2583356218070538],
                    [0.22229355425784175],
                    [0.20512449590031978],
                    [0.19110646351938845]]                    

In [84]:

Rot_list_2_p =  [[np.array([[0.998697, -0.002551, -0.050963],
                            [0.001603, 0.999825, -0.018633],
                            [0.051002, 0.018527, 0.998527]])],
                [np.array([[0.998703, 0.002885, -0.050841],
                            [-0.002291, 0.999929, 0.011736],
                            [0.050871, -0.011604, 0.998638]])],
                [np.array([[0.998223, 0.002014, -0.059548],
                            [-0.001868, 0.999995, 0.002518],
                            [0.059553, -0.002403, 0.998222]])],
                [np.array([[0.998190, 0.004327, -0.059977],
                            [-0.003917, 0.999968, 0.006952],
                            [0.060005, -0.006705, 0.998176]])],
                [np.array([[0.998086, 0.003499, -0.061748],
                            [-0.003063, 0.999970, 0.007141],
                            [0.061772, -0.006939, 0.998066]])],
                [np.array([[0.998066, 0.004304, -0.062017],
                            [-0.003875, 0.999968, 0.007041],
                            [0.062045, -0.006787, 0.998050]])]]


Rot_list_3_p =  [[np.array([[ 9.97793158e-01,  6.38714110e-03, -6.60909825e-02],
                            [-6.44880157e-03,  9.99978947e-01, -7.19666375e-04],
                            [ 6.60849945e-02,  1.14428582e-03,  9.97813341e-01]])],
                [np.array([[ 9.96275246e-01,  6.51261201e-03, -8.59838383e-02],
                            [-6.42488023e-03,  9.99978519e-01,  1.29702253e-03],
                            [ 8.59904383e-02, -7.39755581e-04,  9.96295688e-01]])],
                [np.array([[ 9.96785974e-01,  6.32562526e-03, -7.98605583e-02],
                            [-6.28871776e-03,  9.99979971e-01,  7.13655225e-04],
                            [ 7.98634731e-02, -2.09141007e-04,  9.96805789e-01]])],
                [np.array([[ 9.97862224e-01,  5.95414270e-03, -6.50809498e-02],
                            [-5.88375907e-03,  9.99981880e-01,  1.27308988e-03],
                            [ 6.50873507e-02, -8.87447672e-04,  9.97879176e-01]])],
                [np.array([[0.997864, 0.006056, -0.065046],
                            [-0.006077, 0.999982, -0.000129],
                            [0.065044, 0.000524, 0.997882]])],
                [np.array([[0.997695, 0.005950, -0.067591],
                            [-0.005967, 0.999982, -0.000053],
                            [0.067590, 0.000456, 0.997713]])]]


Rot_list_4_p =  [[np.array([[0.999666, 0.002260, -0.025758],
                            [-0.002284, 0.999997, -0.000921],
                            [0.025756, 0.000979, 0.999668]])],
                [np.array([[0.998905, 0.007131, -0.046242],
                            [-0.009381, 0.998773, -0.048623],
                            [0.045838, 0.049004, 0.997746]])],
                [np.array([[0.997706, 0.006117, -0.067425],
                            [-0.007153, 0.999860, -0.015136],
                            [0.067323, 0.015583, 0.997610]])],
                [np.array([[0.998166, 0.005815, -0.060255],
                            [-0.005915, 0.999981, -0.001477],
                            [0.060246, 0.001831, 0.998182]])],
                [np.array([[0.998106, 0.006102, -0.061222],
                            [-0.006055, 0.999981, 0.000960],
                            [0.061227, -0.000587, 0.998124]])],
                [np.array([[0.998134, 0.005656, -0.060797],
                            [-0.006071, 0.999959, -0.006651],
                            [0.060757, 0.007008, 0.998128]])]]

In [85]:
Trns_list_2_p =  [[np.array([[-482.925694],
                            [-14.473777],
                            [-785.451905]])],
                 [np.array([[-488.563277],
                            [-4.511930],
                            [202.153213]])],
                 [np.array([[-482.972503],
                            [-2.045827],
                            [-45.648413]])],
                 [np.array([[-476.456279],
                            [-0.302435],
                            [-33.620097]])],
               [np.array([[-480.360752],
                            [-0.452980],
                            [-33.557328]])],
               [np.array([[-474.117890],
                            [0.158841],
                            [-34.472694]])]]

Trns_list_3_p =  [[np.array([[-470.30484852],
                            [   1.12431283],
                            [ -24.95595177]])],
                 [np.array([[-473.01852902],
                            [   0.81341954],
                            [ -22.17573026]])],
                 [np.array([[-474.06210272],
                            [   1.62292259],
                            [ -21.70802313]])],
                 [np.array([[-4.74675869e+02],
                            [ 3.59611944e-01],
                            [-1.90589588e+01]])],
               [np.array([[-476.675004],
                            [0.583177],
                            [-15.308127]])],
               [np.array([[-476.303097],
                            [0.952368],
                            [-18.988435]])]]

Trns_list_4_p =  [[np.array([[-485.812480],
                            [-10.814819],
                            [448.155227]])],
                 [np.array([[-489.744803],
                            [10.434341],
                            [-51.906103]])],
                 [np.array([[-458.916068],
                            [-10.625318],
                            [55.431489]])],
                 [np.array([[-432.318535],
                            [-4.379498],
                            [110.675815]])],
               [np.array([[-416.329781],
                            [-10.541641],
                            [112.093923]])],
               [np.array([[-430.929996],
                            [2.312992],
                            [-380.822166]])]]

In [86]:
P_L_2_list_p = []
P_R_2_list_p = []

for i in range(len(ret_R_list_2_p)):
    R1,R2,P_L_est,P_R_est,Q,roi_left,roi_right = cv2.stereoRectify(new_mtxL_list_2_p[i][0], ret_L_list_2_p[i][0], new_mtxR_list_2_p[i][0], ret_R_list_2_p[i][0], (image_size[0],image_size[1]), Rot_list_2_p[i][0], Trns_list_2_p[i][0],flags = cv2.CALIB_ZERO_DISPARITY)
    P_L_2_list_p.append(P_L_est)
    P_R_2_list_p.append(P_R_est)

P_L_3_list_p = []
P_R_3_list_p = []

for i in range(len(ret_R_list_3_p)):
    R1,R2,P_L_est,P_R_est,Q,roi_left,roi_right = cv2.stereoRectify(new_mtxL_list_3_p[i][0], ret_L_list_3_p[i][0], new_mtxR_list_3_p[i][0], ret_R_list_3_p[i][0], (image_size[0],image_size[1]), Rot_list_3_p[i][0], Trns_list_3_p[i][0],flags = cv2.CALIB_ZERO_DISPARITY)
    P_L_3_list_p.append(P_L_est)
    P_R_3_list_p.append(P_R_est)

P_L_4_list_p = []
P_R_4_list_p = []

for i in range(len(ret_R_list_4_p)):
    R1,R2,P_L_est,P_R_est,Q,roi_left,roi_right = cv2.stereoRectify(new_mtxL_list_4_p[i][0], ret_L_list_4_p[i][0], new_mtxR_list_4_p[i][0], ret_R_list_4_p[i][0], (image_size[0],image_size[1]), Rot_list_4_p[i][0], Trns_list_4_p[i][0],flags = cv2.CALIB_ZERO_DISPARITY)
    P_L_4_list_p.append(P_L_est)
    P_R_4_list_p.append(P_R_est)


In [87]:
x_arr_L, X_arr_L, E_L, K_L,P_L = get_image_points(X,PX,PY,thetax=THETA_X_L,thetay = THETA_Y_L,thetaz = THETA_Z_L,trans_x= -C_L[0],trans_y= -C_L[1],trans_z= -C_L[2],F = F,mx = (1/pixel_width),my =(1/pixel_width))
x_arr_R, X_arr_R, E_R, K_R,P_R = get_image_points(X,PX,PY,thetax = THETA_X_R,thetay = THETA_Y_R, thetaz = THETA_Z_R,trans_x= -C_R[0],trans_y= -C_R[1],trans_z= -C_R[2],F = F,mx = (1/pixel_width),my =(1/pixel_width))

In [88]:
n_distance = len(zdistance)
len_chess = len(ret_L_list_2_p)

XestOpenCV_2 = np.zeros((len_chess,n_distance,4,1))
Xoriginal_2 = np.zeros((len_chess,n_distance,4,1))

Error_2_abs_Z = np.zeros((len_chess,n_distance,1)) 
relative_Error_2_Z = np.zeros((len_chess,n_distance,1)) 

XestOpenCV_3 = np.zeros((len_chess,n_distance,4,1))
Xoriginal_3 = np.zeros((len_chess,n_distance,4,1))

XestOpenCV_4 = np.zeros((len_chess,n_distance,4,1))
Xoriginal_4 = np.zeros((len_chess,n_distance,4,1))

estError_2 = np.zeros((len_chess,n_distance,1))
estError_3 = np.zeros((len_chess,n_distance,1))
estError_4 = np.zeros((len_chess,n_distance,1))

X_est_arr_2 = np.zeros((len_chess,n_distance,4,1))
X_org_arr_2 = np.zeros((len_chess,n_distance,4,1))

X_est_arr_3 = np.zeros((len_chess,n_distance,4,1))
X_org_arr_3 = np.zeros((len_chess,n_distance,4,1))

X_est_arr_4 = np.zeros((len_chess,n_distance,4,1))
X_org_arr_4 = np.zeros((len_chess,n_distance,4,1))

triangulation_error_2_p =np.zeros((len_chess,1))
triangulation_error_3_p =np.zeros((len_chess,1))
triangulation_error_4_p =np.zeros((len_chess,1))


for i in range(len(ret_R_list_2_p)):
    for j in range(len(centroid_L_arr)):
        # 2 meters
        XestOpenCV_2[i,j,:,:] = cv2.triangulatePoints(P_L_2_list_p[i],P_R_2_list_p[i], centroid_L_arr[j], centroid_R_arr[j])
        Xoriginal_2[i,j,:,:] =  cv2.triangulatePoints(P_L_original_p[i],P_R_original_p[i],x_arr_L[j], x_arr_R[j])

        X_est_arr_2[i,j,:,:] = XestOpenCV_2[i,j,:,:]/XestOpenCV_2[i,j,-1,:]
        X_org_arr_2[i,j,:,:] = Xoriginal_2[i,j,:,:]/Xoriginal_2[i,j,-1,:]

        estError_2[i,j,:] = np.sqrt(np.sum(np.square(X_est_arr_2[i,j,:3,:]-X_org_arr_2[i,j,:3,:])))

        Error_2_abs_Z[i,j,:] = np.abs(X_est_arr_2[i,j,2,0]-X_org_arr_2[i,j,2,0])
        relative_Error_2_Z[i,j,:] = np.abs(X_est_arr_2[i,j,2,0]-X_org_arr_2[i,j,2,0])/X_org_arr_2[i,j,2,0]*100
        
        # 3 meters
        XestOpenCV_3[i,j,:,:] = cv2.triangulatePoints(P_L_3_list_p[i],P_R_3_list_p[i], centroid_L_arr[j], centroid_R_arr[j])
        Xoriginal_3[i,j,:,:] =  cv2.triangulatePoints(P_L_original_p[i],P_R_original_p[i],x_arr_L[j], x_arr_R[j])

        X_est_arr_3[i,j,:,:] = XestOpenCV_3[i,j,:,:]/XestOpenCV_3[i,j,-1,:]
        X_org_arr_3[i,j,:,:] = Xoriginal_3[i,j,:,:]/Xoriginal_3[i,j,-1,:]

        estError_3[i,j,:] = np.sqrt(np.sum(np.square(X_est_arr_3[i,j,:3,:]-X_org_arr_3[i,j,:3,:])))
        # 4 meters
        XestOpenCV_4[i,j,:,:] = cv2.triangulatePoints(P_L_4_list_p[i],P_R_4_list_p[i], centroid_L_arr[j], centroid_R_arr[j])
        Xoriginal_4[i,j,:,:] =  cv2.triangulatePoints(P_L_original_p[i],P_R_original_p[i],x_arr_L[j], x_arr_R[j])

        X_est_arr_4[i,j,:,:] = XestOpenCV_4[i,j,:,:]/XestOpenCV_4[i,j,-1,:]
        X_org_arr_4[i,j,:,:] = Xoriginal_4[i,j,:,:]/Xoriginal_4[i,j,-1,:]

        estError_4[i,j,:] = np.sqrt(np.sum(np.square(X_est_arr_4[i,j,:3,:]-X_org_arr_4[i,j,:3,:])))

    triangulation_error_2_p[i,:] = np.sum(estError_2[i,:,:])/estError_2[i,:,:].shape[0]
    triangulation_error_3_p[i,:] = np.sum(estError_3[i,:,:])/estError_3[i,:,:].shape[0]
    triangulation_error_4_p[i,:] = np.sum(estError_4[i,:,:])/estError_4[i,:,:].shape[0]


In [89]:
i = 3
j = 300

print(X_est_arr_2[i,j,2,0])
print(X_org_arr_2[i,j,2,0])

print(np.abs(X_est_arr_2[i,j,2,0]-X_org_arr_2[i,j,2,0]))

print(np.abs(X_est_arr_2[i,j,2,0]-X_org_arr_2[i,j,2,0])/X_org_arr_2[i,j,2,0]*100)

309039.24525548436
299999.9992724972
9039.245982987166
3.0130820016358073


In [100]:
i = 3
j = 3



2999.999992725

In [97]:

i = 3
j = 3

np.abs(X_est_arr_2[i,j,2,0]-X_org_arr_2[i,j,2,0])/X_org_arr_2[i,j,2,0]*100


9.451508742469914

In [91]:
np.save("E_err2_p_centroid_infi_round.npy",Error_2_abs_Z)
np.save("relative_Error_2_Z_centroid_infi_round.npy",relative_Error_2_Z)
np.save("X_org_arr_2_centroid_infi.npy",X_org_arr_2)
np.save("X_est_arr_2_centroid_infi.npy",X_est_arr_2)

In [ ]:
#estError_20_distance
E_err2_p = np.zeros((estError_2.shape[0],n_distance))
E_err3_p = np.zeros((estError_3.shape[0],n_distance))
E_err4_p = np.zeros((estError_4.shape[0],n_distance))
for j in range(estError_2.shape[0]):
    for i in range(estError_2.shape[1]):
        E_err2_p[j,i] = np.sum(np.abs(X_est_arr_2[j,i,:3,:]-X_org_arr_2[j,i,:3,:]))/(np.sum(X_est_arr_2[j,i,:3,:]))*100
        E_err3_p[j,i] = np.sum(np.abs(X_est_arr_3[j,i,:3,:]-X_org_arr_3[j,i,:3,:]))/(np.sum(X_est_arr_3[j,i,:3,:]))*100
        E_err4_p[j,i] = np.sum(np.abs(X_est_arr_4[j,i,:3,:]-X_org_arr_4[j,i,:3,:]))/(np.sum(X_est_arr_4[j,i,:3,:]))*100

In [ ]:
fig = plt.figure(figsize = (20,10))
ax = fig.add_subplot()

x_marker = np.linspace(10e-10,1e3,1000)
y_marker = np.ones((1000,1))
#ax.plot(x_marker,y_marker*0.001,'b--')

color_2 = ['black','firebrick','gold','chartreuse','darkcyan','m']
sub_2 = ["2 meters with 5 chessboards","2 meters with 20 chessboards","2 meters with 45 chessboards","2 meters with 80 chessboards","2 meters with 125 chessboards","2 meters with 180 chessboards"]
color_3 = ['red','g','orange','c','darkblue','fuchsia']
sub_3 = ["3 meters with 5 chessboards","3 meters with 20 chessboards","3 meters with 45 chessboards","3 meters with 80 chessboards","3 meters with 125 chessboards","3 meters with 180 chessboards"]
color_4 = ['darkslategray','brown','tomato','aqua','lime','y']
sub_4 = ["4 meters with 5 chessboards","4 meters with 20 chessboards","4 meters with 45 chessboards","4 meters with 80 chessboards","4 meters with 125 chessboards","4 meters with 180 chessboards"]

for i in range(estError_2.shape[0]):

    ax.plot(zdistance/1000,E_err2_p[i,:],color =  color_2[i], label = sub_2[i], marker = '^')
    #ax.plot(zdistance/1000,E_err3_p[i,:],color = color_3[i],label = sub_3[i], marker = '*')
    ax.plot(zdistance/1000,E_err3_p[i,:],color = color_3[i],label = sub_3[i], marker = 'p')
    ax.plot(zdistance/1000,E_err4_p[i,:],color = color_4[i],label = sub_4[i], marker = 's')
    #ax.plot(zdistance/1000,E_err4_p,'--*',label = "2 meters with subpixelgrid = 64")
    #ax.plot(zdistance/1000,E_err2_pixel,'b-*', label = "2 meters pixelixed version")
    #ax.plot(zdistance/1000,E_err20_p,'g-*',label = "2 meters with subpixelgrid = 128")

ax.set_yscale('log')


ax.set_xlim(0,50)
ax.set_ylabel("Triangulation errror [%]")
ax.set_title("Triangulation error")
ax.set_xlabel("Distance [m]")
ax.legend()
ax.grid()

In [ ]:
# number of chessboards
nr_chess = [5,20,45,80,125,180]


fig = plt.figure(figsize = (20,10))
ax = fig.add_subplot(211)
ax2 = fig.add_subplot(212)

x_marker = np.linspace(10e-10,1e3,1000)
y_marker = np.ones((1000,1))
#ax.plot(x_marker,y_marker*0.001,'b--')

color_2 = ['black','firebrick','gold','chartreuse','darkcyan','m']
sub_2 = ["2 meters with 5 chessboards w/ centroding","2 meters with 20 chessboards w/ centroding","2 meters with 45 chessboards w/ centroding","2 meters with 80 chessboards w/ centroding","2 meters with 125 chessboards w/ centroding","2 meters with 180 chessboards w/ centroding"]
color_3 = ['red','g','orange','c','darkblue','fuchsia']
sub_3 = ["3 meters with 5 chessboards w/ centroding","3 meters with 20 chessboards w/ centroding","3 meters with 45 chessboards w/ centroding","3 meters with 80 chessboards w/ centroding","3 meters with 125 chessboards w/ centroding","3 meters with 180 chessboards w/ centroding"]
color_4 = ['darkslategray','brown','tomato','aqua','lime','y']
sub_4 = ["4 meters with 5 chessboards w/ centroding","4 meters with 20 chessboards w/ centroding","4 meters with 45 chessboards w/ centroding","4 meters with 80 chessboards w/ centroding","4 meters with 125 chessboards w/ centroding","4 meters with 180 chessboards w/ centroding"]

for i in range(X_org_arr_2.shape[0]):

    ax.plot(zdistance[:50]/1000,X_est_arr_2[i,:50,2]/1000,color =  color_2[i] , marker = '^', label = sub_2[i])
    ax.plot(zdistance[:50]/1000,X_est_arr_3[i,:50,2]/1000,color =  color_3[i] , marker = '^', label = sub_3[i])
    ax.plot(zdistance[:50]/1000,X_est_arr_4[i,:50,2]/1000,color =  color_4[i] , marker = '^', label = sub_4[i])
    ax.plot(zdistance[:50]/1000,zdistance[:50]/1000,color = 'b',label = "1:1 correspondence", marker = '*')

ax2.plot(nr_chess,triangulation_error_2_p/1000,color = 'b'  , marker = '^',linestyle = 'dashed', label = sub_2[i])
ax2.plot(nr_chess,triangulation_error_3_p/1000,color =  'g' , marker = '^',linestyle = 'dashed', label = sub_3[i])
ax2.plot(nr_chess,triangulation_error_4_p/1000,color =  'k' , marker = '^',linestyle = 'dashed', label = sub_4[i])

    
ax2.set_yscale('log')


ax.set_ylabel("Triangulation distance[m] ")
ax.set_title("Triangulation distance")
ax.set_xlabel("Distance [m]")
#ax.legend()
ax.grid()
ax2.grid()